In [32]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import statsmodels.api as sm

dataset = pd.read_csv('financial_fraud_detection_dataset.csv')

In [24]:
# Tidying dataset 
dataset = dataset.drop(['transaction_id', 'timestamp', 'sender_account', 'receiver_account', 'fraud_type', 'time_since_last_transaction', 'ip_address', 'device_hash'], axis=1)

# Display all the columns of the dataset 
dataset.head(5)

,amount,transaction_type,merchant_category,location,device_used,is_fraud,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel
0,343.78,withdrawal,utilities,Tokyo,mobile,False,-0.21,3,0.22,card
1,419.65,withdrawal,online,Toronto,atm,False,-0.14,7,0.96,ACH
2,2773.86,deposit,other,London,pos,False,-1.78,20,0.89,card
3,1666.22,deposit,online,Sydney,pos,False,-0.60,6,0.37,wire_transfer
4,24.43,transfer,utilities,Toronto,mobile,False,0.79,13,0.27,ACH


## Introduction 

Financial fraud  happens when someone takes money or other assets from you illegally, and there are many different types types such as identity, payment, account takeover, etc. Especially due to the growing banking system and the financial technologies, we would like to research and learn more into what could be used to determine whether a transaction is fraud or not, and which category(ies) would be more important than others for detection.

Our goal for this project is to detect financial fraud using machine learning given synthetical data. We would like to be able to detect whether a transaction is fraud or not, and what type of fraud it would be.

## Variable Description

There were a total of 18 variables in the original dataset which we had removed for 10. 

The following variables are the ones we will focus on: 

- amount (USD)
- transaction_type: type of transaction [deposit,  withdrawal, transfer, payment]
- merchant_category: business category involved in transaction [retail, travel, online, utilities, entertainment, etc...]
- location: location from where transaction was initiated
- device_used: device type used [mobile, web, atm, pos]
- is_fraud: boolean flag indicating whether the transaction was fraudulent [true, false]
- spending_deviation_score: deviation from normal spending pattern
- velocity_score: number of transactions over a recent period
- geo_anomaly_score: measure of unusual geographic transaction behaviour [0, 1]
- payment_channel: card, ACH, wire_transfer, UPI


In [25]:
# We want to find the correlation between the features and the target variable 'is_fraud'
dataset = pd.get_dummies(dataset, columns=['merchant_category', 'location', 'payment_channel', 'device_used', 'transaction_type'])

To improve model clarity, we can first remove a few irrelevant variables first. The variables transaction_id, timestamp, sender_account, receiver_account, ip_address and device_hash provide arbitriary technical details, making it unlikely for these variables to be as helpful for detecting a fraud transaction.

Linear regression relies on numerial data as it uses mathematical operations, like addition, subtraction, etc., to fit a linear equation to observed data. However, this dataset has multiple categorical variables, so we first need to make these into indicator variables. This is essential, as it converts categories into a binary variable, making linear regression possible with these variables. We will use the `get-dummies` function from the Pandas library to do this.

In [30]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset.drop('is_fraud', axis=1), dataset['is_fraud'], test_size=0.3, random_state=42)

# Fit the Logistic Regression model 
clf = LogisticRegression(max_iter=10000)
clf.fit(X_train, y_train) 

# Feature selection using RFE 
rfe = RFE(clf) 
rfe.fit(X_train, y_train)

# Get the ranking of features
feature_ranking = pd.DataFrame({'Feature': X_train.columns, 'Ranking': rfe.ranking_}) 
# Sort the features by their ranking
feature_ranking = feature_ranking.sort_values(by='Ranking')
# Display the feature ranking
print(feature_ranking)


                            Feature  Ranking
31      transaction_type_withdrawal        1
30        transaction_type_transfer        1
14                  location_London        1
13                   location_Dubai        1
18                   location_Tokyo        1
20              payment_channel_ACH        1
21              payment_channel_UPI        1
16               location_Singapore        1
22             payment_channel_card        1
24                  device_used_atm        1
25               device_used_mobile        1
26                  device_used_pos        1
27                  device_used_web        1
28         transaction_type_deposit        1
29         transaction_type_payment        1
23    payment_channel_wire_transfer        1
12                  location_Berlin        2
17                  location_Sydney        3
15                location_New York        4
19                 location_Toronto        5
11      merchant_category_utilities        6
8      mer